# Testing
### Viertelstundenwerte zur Stromabgabe nach Netzebenen in der Stadt Zürich, seit ?
Datum: 04.10.2022

Diesen Winter wird ein Engergie-Enpass erwartet. Daher ist es wichtig, zeitnah beobachten zu können, wie sich der Energieverbrauch entwickelt. Hier wird Fokus auf Stromabgabe nach Grosskunden und Privathaushalten gelegt. 

Dieses Skript dient vorerst zum Testen der Datenlieferungen, welche ähnlich wie die Bruttolastgangdaten geliefert werden.



### Importiere die notwendigen Packages

In [13]:
#%pip install geopandas altair fiona requests folium mplleaflet contextily seaborn datetime plotly leafmap

In [14]:
import os
import glob
import io

import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui
import numpy as np
import altair as alt
import matplotlib.pyplot as plt

import datetime
import geopandas as gpd
import folium 
import plotly.express as px
import seaborn as sns


import requests


import plotly.express as px


In [15]:
SSL_VERIFY = False
# evtl. SSL_VERIFY auf False setzen wenn die Verbindung zu https://www.gemeinderat-zuerich.ch nicht klappt (z.B. wegen Proxy)
# Um die SSL Verifikation auszustellen, bitte die nächste Zeile einkommentieren ("#" entfernen)
# SSL_VERIFY = False

In [16]:
if not SSL_VERIFY:
    import urllib3
    urllib3.disable_warnings()

Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [17]:
#pd.options.display.float_format = lambda x : '{:,.1f}'.format(x) if (np.isnan(x) | np.isinf(x)) else '{:,.0f}'.format(x) if int(x) == x else '{:,.1f}'.format(x)
pd.options.display.float_format = '{:.5f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

### Neu: Datenaufbereitung

Die Daten werden täglich auf die INT_EWZ-Dropzone gestellt. In diesem Schritt sollen alle csv im Verzeichnis eingelesen und nach Datum sortiert werden.

Anfangs 2020 gabs noch ein durcheinander mit Monatsfiles und Tagesfiles, die sich kreuzen. Beim Import finden dann wohl doppelte Einträge statt.


In [18]:
import os


In [19]:
yourpath = r"\\szh.loc\ssz\applikationen\OGD_Dropzone\INT_EWZ\ewz_stromabgabe_netzebenen_stadt_zuerich"

column_names = ["timestamp", "strom_ne5", "status_ne5", "strom_ne7", "status_ne7"]
dfs = []

for root, dirs, files in os.walk(yourpath, topdown=True):
    for name in files:
        print(root, name)
        # read the csv file
        if name.endswith('.csv'):
            dfs.append(pd.read_csv(
                os.path.join(root, name)
                , sep=';'
                , skiprows= 5
                , encoding='latin-1'
                , header = None
                , names = column_names
                , decimal='.'
                , parse_dates=[0]
                , dayfirst=True
            ))
        else:
            print("skip: "+name)
print('done')       
dfs

\\szh.loc\ssz\applikationen\OGD_Dropzone\INT_EWZ\ewz_stromabgabe_netzebenen_stadt_zuerich UW_ZH_Summiert_Azure_20220925 bis 20220926_003.csv
\\szh.loc\ssz\applikationen\OGD_Dropzone\INT_EWZ\ewz_stromabgabe_netzebenen_stadt_zuerich vorlage_ewz_bruttolastgang_stadt_zuerich_metadata.xlsx
skip: vorlage_ewz_bruttolastgang_stadt_zuerich_metadata.xlsx
done


[             timestamp   strom_ne5 status_ne5   strom_ne7 status_ne7
 0  2022-09-25 00:15:00 20720.87740          F 42686.87260          F
 1  2022-09-25 00:30:00 20568.84920          F 41912.35080          F
 2  2022-09-25 00:45:00 20093.10820          F 40288.49180          F
 3  2022-09-25 01:00:00 19749.14350          F 38904.10650          F
 4  2022-09-25 01:15:00 19118.30610          F 38189.49390          F
 ..                 ...         ...        ...         ...        ...
 91 2022-09-25 23:00:00 20258.65420          F 42610.50040          F
 92 2022-09-25 23:15:00 19911.55950          F 41277.82490          F
 93 2022-09-25 23:30:00 19894.38420          F 40044.96410          F
 94 2022-09-25 23:45:00 19949.04060          F 38749.96670          F
 95 2022-09-26 00:00:00 20282.57830          F 37657.93310          F
 
 [96 rows x 5 columns]]

In [20]:
# Concatenate all data into one DataFrame
df_stromabgabe = pd.concat(dfs, ignore_index=False)

In [21]:
df_stromabgabe.sort_values('timestamp', ascending=False)
df_stromabgabe.head(2)
df_stromabgabe.dtypes

timestamp     datetime64[ns]
strom_ne5            float64
status_ne5            object
strom_ne7            float64
status_ne7            object
dtype: object

In [22]:
#Formatiere die Datumswerte nach ISO-Standard (wird jedoch zu Char-Objekt...)
df_stromabgabe['timestamp'] = df_stromabgabe.timestamp.apply(datetime.datetime.isoformat)

In [23]:
df_stromabgabe

,timestamp,strom_ne5,status_ne5,strom_ne7,status_ne7
0,2022-09-25T00:15:00,20720.87740,F,42686.87260,F
1,2022-09-25T00:30:00,20568.84920,F,41912.35080,F
2,2022-09-25T00:45:00,20093.10820,F,40288.49180,F
3,2022-09-25T01:00:00,19749.14350,F,38904.10650,F
4,2022-09-25T01:15:00,19118.30610,F,38189.49390,F
...,...,...,...,...,...
91,2022-09-25T23:00:00,20258.65420,F,42610.50040,F
92,2022-09-25T23:15:00,19911.55950,F,41277.82490,F
93,2022-09-25T23:30:00,19894.38420,F,40044.96410,F
94,2022-09-25T23:45:00,19949.04060,F,38749.96670,F


Berechne weitere Attribute falls notwendig

In [210]:
data2betested = (
    df_stromabgabe
    .copy()
    .assign(
        #Aktualisierungs_Datum_str= lambda x: x.Aktualisierungs_Datum.astype(str),
        timestamp_str = lambda x: x.timestamp.astype(str),
        timestamp_str_ISO = lambda x: x.timestamp.dt.strftime("%Y-%m-%dT%H:%M:%S"),        
        day = lambda x: x.timestamp.dt.strftime("%Y-%m-%d"),
        day_of_year = lambda x: x.timestamp.dt.day_of_year,      
        day_nr = lambda x: x.timestamp.dt.day,
        weekday = lambda x: x.timestamp.dt.dayofweek,
        weekday_name = lambda x: x.timestamp.dt.day_name(),
        year = lambda x: x.timestamp.dt.year,
        month = lambda x: x.timestamp.dt.month
    )
    .sort_values(['status_ne5','timestamp'], ascending=[False, False])
    )
data2betested.columns
#data2betested.dtypes
data2betested

,timestamp,strom_ne5,status_ne5,strom_ne7,status_ne7,timestamp_str,timestamp_str_ISO,day,day_of_year,day_nr,weekday,weekday_name,year,month
95,2022-09-26 00:00:00,20282.57830,F,37657.93310,F,2022-09-26 00:00:00,2022-09-26T00:00:00,2022-09-26,269,26,0,Monday,2022,9
94,2022-09-25 23:45:00,19949.04060,F,38749.96670,F,2022-09-25 23:45:00,2022-09-25T23:45:00,2022-09-25,268,25,6,Sunday,2022,9
93,2022-09-25 23:30:00,19894.38420,F,40044.96410,F,2022-09-25 23:30:00,2022-09-25T23:30:00,2022-09-25,268,25,6,Sunday,2022,9
92,2022-09-25 23:15:00,19911.55950,F,41277.82490,F,2022-09-25 23:15:00,2022-09-25T23:15:00,2022-09-25,268,25,6,Sunday,2022,9
91,2022-09-25 23:00:00,20258.65420,F,42610.50040,F,2022-09-25 23:00:00,2022-09-25T23:00:00,2022-09-25,268,25,6,Sunday,2022,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2022-09-25 01:15:00,19118.30610,F,38189.49390,F,2022-09-25 01:15:00,2022-09-25T01:15:00,2022-09-25,268,25,6,Sunday,2022,9
3,2022-09-25 01:00:00,19749.14350,F,38904.10650,F,2022-09-25 01:00:00,2022-09-25T01:00:00,2022-09-25,268,25,6,Sunday,2022,9
2,2022-09-25 00:45:00,20093.10820,F,40288.49180,F,2022-09-25 00:45:00,2022-09-25T00:45:00,2022-09-25,268,25,6,Sunday,2022,9
1,2022-09-25 00:30:00,20568.84920,F,41912.35080,F,2022-09-25 00:30:00,2022-09-25T00:30:00,2022-09-25,268,25,6,Sunday,2022,9


Eine elegantere Möglichkeit wäre so wie hier vorzugehen. Da stimmt aber etwas mit os.listdir() nicht... muss ich noch prüfen.

    yourpath = r"\\szh.loc\ssz\applikationen\OGD_Dropzone\INT_EWZ\ewz_bruttolastgang_stadt_zuerich"
    #yourpath = r"P:\OGD_Dropzone\INT_EWZ\ewz_bruttolastgang_stadt_zuerich"

    def f(i):
        return pd.read_csv(i, header=None, sep=';', skiprows= 5, encoding='latin-1', decimal=',', parse_dates=[0]
            )
    filepaths = [f for f in os.listdir(yourpath) if f.endswith('.csv')]
    df = pd.concat(map(f, filepaths))


### Verwende das Datum als Index

While we did already parse the `datetime` column into the respective datetime type, it currently is just a regular column. 
**To enable quick and convenient queries and aggregations, we need to turn it into the index of the DataFrame**

In [211]:
data2betested = data2betested.set_index("timestamp")
#df_bruttolastgang = df_bruttolastgang.set_index("zeitpunkt")

In [212]:
data2betested.info()
data2betested.index.year.unique()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 96 entries, 2022-09-26 00:00:00 to 2022-09-25 00:15:00
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   strom_ne5          96 non-null     float64
 1   status_ne5         96 non-null     object 
 2   strom_ne7          96 non-null     float64
 3   status_ne7         96 non-null     object 
 4   timestamp_str      96 non-null     object 
 5   timestamp_str_ISO  96 non-null     object 
 6   day                96 non-null     object 
 7   day_of_year        96 non-null     int64  
 8   day_nr             96 non-null     int64  
 9   weekday            96 non-null     int64  
 10  weekday_name       96 non-null     object 
 11  year               96 non-null     int64  
 12  month              96 non-null     int64  
dtypes: float64(2), int64(5), object(6)
memory usage: 10.5+ KB


Int64Index([2022], dtype='int64', name='timestamp')

In [213]:
data2betested.sort_values("timestamp",ascending=False)

,strom_ne5,status_ne5,strom_ne7,status_ne7,timestamp_str,timestamp_str_ISO,day,day_of_year,day_nr,weekday,weekday_name,year,month
timestamp,,,,,,,,,,,,,
2022-09-26 00:00:00,20282.57830,F,37657.93310,F,2022-09-26 00:00:00,2022-09-26T00:00:00,2022-09-26,269,26,0,Monday,2022,9
2022-09-25 23:45:00,19949.04060,F,38749.96670,F,2022-09-25 23:45:00,2022-09-25T23:45:00,2022-09-25,268,25,6,Sunday,2022,9
2022-09-25 23:30:00,19894.38420,F,40044.96410,F,2022-09-25 23:30:00,2022-09-25T23:30:00,2022-09-25,268,25,6,Sunday,2022,9
2022-09-25 23:15:00,19911.55950,F,41277.82490,F,2022-09-25 23:15:00,2022-09-25T23:15:00,2022-09-25,268,25,6,Sunday,2022,9
2022-09-25 23:00:00,20258.65420,F,42610.50040,F,2022-09-25 23:00:00,2022-09-25T23:00:00,2022-09-25,268,25,6,Sunday,2022,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-25 01:15:00,19118.30610,F,38189.49390,F,2022-09-25 01:15:00,2022-09-25T01:15:00,2022-09-25,268,25,6,Sunday,2022,9
2022-09-25 01:00:00,19749.14350,F,38904.10650,F,2022-09-25 01:00:00,2022-09-25T01:00:00,2022-09-25,268,25,6,Sunday,2022,9
2022-09-25 00:45:00,20093.10820,F,40288.49180,F,2022-09-25 00:45:00,2022-09-25T00:45:00,2022-09-25,268,25,6,Sunday,2022,9


In [214]:
data2betested.loc["2022-08-12"]

,strom_ne5,status_ne5,strom_ne7,status_ne7,timestamp_str,timestamp_str_ISO,day,day_of_year,day_nr,weekday,weekday_name,year,month
timestamp,,,,,,,,,,,,,


### Zeitvariabeln
Bestimme den aktuellst geladenen Monat. Hier ist es der Stand vor 2 Monaten. 
Bestimme noch weitere evt. sinnvolle Zeitvariabeln.

Zum Unterschied zwischen import datetime und from datedtime import datetime, siehe https://stackoverflow.com/questions/15707532/import-datetime-v-s-from-datetime-import-datetime

Zuerst die Zeitvariabeln als Strings

In [215]:
#today_date = datetime.date.today()
#date_time = datetime.datetime.strptime(date_time_string, '%Y-%m-%d %H:%M')
now = datetime.date.today()
date_today = now.strftime("%Y-%m-%d")
year_today = now.strftime("%Y")
month_today = now.strftime("%m")
day_today = now.strftime("%d")

date_day_a_week_ago = (datetime.datetime.now() - datetime.timedelta(days=7)).date()
day_a_week_ago = date_day_a_week_ago.strftime('%Y-%m-%d')


print(now," vor einer Woche: ", day_a_week_ago)


2022-10-04  vor einer Woche:  2022-09-27


Und hier noch die Zeitvariabeln als Integers:
- `aktuellesJahr`
- `aktuellerMonat`: Der gerade jetzt aktuelle Monat
- `selectedMonat`: Der aktuellste Monat in den Daten. In der Regel zwei Monate her.

In [216]:
#now = datetime.now() 
int_times = now.timetuple()

aktuellesJahr = int_times[0]
aktuellerMonat = int_times[1]
selectedMonat = int_times[1]-2

print(aktuellesJahr, 
      aktuellerMonat,
    'datenstand: ', selectedMonat,
     int_times)


2022 10 datenstand:  8 time.struct_time(tm_year=2022, tm_mon=10, tm_mday=4, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=1, tm_yday=277, tm_isdst=-1)


In [217]:
date_today = now.strftime("%Y-%m-%d")
year_today = now.strftime("%Y")
month_today = now.strftime("%m")
day_today = now.strftime("%d")

### Einfache Datentests

 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [218]:
#data2betested.head(6)

In [219]:
#data2betested.dtypes

Beschreibe einzelne Attribute

In [220]:
data2betested.describe()

,strom_ne5,strom_ne7,day_of_year,day_nr,weekday,year,month
count,96.00000,96.00000,96.00000,96.00000,96.00000,96.00000,96.00000
mean,20071.03020,44925.02705,268.01042,25.01042,5.93750,2022.00000,9.00000
std,972.81770,7806.11565,0.10206,0.10206,0.61237,0.00000,0.00000
min,17709.11840,32394.71940,268.00000,25.00000,0.00000,2022.00000,9.00000
25%,19675.35525,37213.70920,268.00000,25.00000,6.00000,2022.00000,9.00000
50%,20270.61625,47129.76420,268.00000,25.00000,6.00000,2022.00000,9.00000
75%,20743.42177,51151.63996,268.00000,25.00000,6.00000,2022.00000,9.00000
max,21374.93215,57015.94650,269.00000,26.00000,6.00000,2022.00000,9.00000


Wie viele Nullwerte gibt es im Datensatz?

In [221]:
data2betested.isnull().sum()

strom_ne5            0
status_ne5           0
strom_ne7            0
status_ne7           0
timestamp_str        0
timestamp_str_ISO    0
day                  0
day_of_year          0
day_nr               0
weekday              0
weekday_name         0
year                 0
month                0
dtype: int64

Die Minimumtemperatur scheint nicht zu funktionieren...

In [222]:
#data2betested[np.isnan(data2betested.bruttolastgang)]

### Checke die Stati

Das Attribut Status gibt einen Hinweis dazu:

- Status **W - wahre Werte** gesetzt ist, kann man die Werte als der Realität entsprechend interpretieren. Sie werden nicht mehr angepasst und bleiben stabil.
- Status **F fehlen noch Messwerte** zur korrekten Berechnung des Bruttolastgangs. Die angegebenen Werte werden noch angepasst werden müssen.
- Status **E wurden weitere Messwerte aufgenommen und der Bruttolastgang entsprechend angepasst**. Weitere Anpassungen werden aber noch vorgenommen werden müssen, bis der Status W erreicht werden kann.

In [223]:
df_status = data2betested\
    .groupby(['status_ne5','status_ne7']) \
    .agg(cnt_status_ne5=('status_ne5', 'count'),cnt_status_ne7=('status_ne7', 'count')
        )\
    .sort_values('cnt_status_ne5', ascending=False) 

df_status.reset_index()


,status_ne5,status_ne7,cnt_status_ne5,cnt_status_ne7
0,F,F,96,96


### Einfache Visualisierungen zur Plausi

Exploriere die Daten mit Pivottable.JS

In [226]:
#from pivottablejs import pivot_ui

#pivot_ui(data2betested)

### Zeitpunkte und Zeiträume abfragen

A particular powerful feature of the Pandas DataFrame is its indexing capability that also works using time-based entities, such as dates and times. We have already created the index above, so let's put it to use.

In [230]:
data2betested.loc["2022-09"].head(2)
#data2betested.loc["2021-10-31":"2021-11-30"].head(2)

,strom_ne5,status_ne5,strom_ne7,status_ne7,timestamp_str,timestamp_str_ISO,day,day_of_year,day_nr,weekday,weekday_name,year,month
timestamp,,,,,,,,,,,,,
2022-09-26 00:00:00,20282.57830,F,37657.93310,F,2022-09-26 00:00:00,2022-09-26T00:00:00,2022-09-26,269,26,0,Monday,2022,9
2022-09-25 23:45:00,19949.04060,F,38749.96670,F,2022-09-25 23:45:00,2022-09-25T23:45:00,2022-09-25,268,25,6,Sunday,2022,9


### Visualisierungen nach Zeitausschnitten

In [231]:
data2betested.columns
data2betested.dtypes

strom_ne5            float64
status_ne5            object
strom_ne7            float64
status_ne7            object
timestamp_str         object
timestamp_str_ISO     object
day                   object
day_of_year            int64
day_nr                 int64
weekday                int64
weekday_name          object
year                   int64
month                  int64
dtype: object

#### Entwicklung des Bruttolastgangs

Wöchentliche, tägliche und stündliche Summierungen der Passagierströme

In [232]:
yearly_sums = data2betested[['strom_ne5', 'strom_ne7']].resample("Y").sum()
monthly_sums = data2betested[['strom_ne5', 'strom_ne7']].resample("M").sum()
weekly_sums = data2betested[['strom_ne5', 'strom_ne7']].resample("W").sum()
daily_sums  = data2betested[['strom_ne5', 'strom_ne7']].resample("D").sum()
hourly_sums  = data2betested[['strom_ne5', 'strom_ne7']].resample("H").sum()

#weekly_sums.sort_values('pedestrians_count', ascending=False)
#daily_sums.sort_values('pedestrians_count', ascending=False)
#hourly_sums.sort_values('pedestrians_count', ascending=False)
daily_sums.sort_values('timestamp', ascending=False).head(5)
#monthly_sums.sort_values('pedestrians_count', ascending=False)
yearly_sums.sort_values('timestamp', ascending=False).head(5)
yearly_sums

,strom_ne5,strom_ne7
timestamp,,
2022-12-31,1926818.89910,4312802.59670


In [233]:
data2betested.columns
data2betested.dtypes

strom_ne5            float64
status_ne5            object
strom_ne7            float64
status_ne7            object
timestamp_str         object
timestamp_str_ISO     object
day                   object
day_of_year            int64
day_nr                 int64
weekday                int64
weekday_name          object
year                   int64
month                  int64
dtype: object

In [234]:
#myAgg = data2betested.loc["2008-11-30":"2021-10-31"]
myAgg = data2betested.loc["2019-01-01":date_today].query('status != "W"')\
    .groupby(['status','day','weekday', 'weekday_name', 'year', 'month']) \
    .agg(sum_bruttolastgang =('bruttolastgang', 'sum')
         , cnt_bruttolastgang =('bruttolastgang', 'count')
        , avg_bruttolastgang=('bruttolastgang', 'mean')
        )\
    .sort_values('cnt_bruttolastgang', ascending=True) 

myAgg.reset_index()
#myAgg.reset_index().dtypes

C:\ProgramData\Anaconda3\envs\gogo_gis\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  


UndefinedVariableError: name 'status' is not defined

In [55]:
myTitle="Bruttolastgang Tagessummen seit 2019 (Rohdaten)"

highlight = alt.selection(type='single', on='mouseover',
                          fields=['sum_bruttolastgang'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(myAgg.reset_index().query('status != "W"'), title=myTitle).encode(
    x=alt.X('day:T', axis=alt.Axis(title='Wochentag (0=Mo, 6=So)'))# , axis=alt.Axis(format='%', title='percentage')
    , y=alt.X('avg_bruttolastgang', axis=alt.Axis(title='Bruttolastgang'))
    , color=alt.Color('status', legend=alt.Legend(title="Status", orient="right"))  
    ,tooltip=['status', 'day', 'weekday', 'weekday_name', 'year', 'month', 'sum_bruttolastgang']    
)
points = base.mark_circle().encode(
    opacity=alt.value(0.75)
).add_selection(
    highlight
).properties(
    width=750 , height=350
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(0.5), alt.value(4))
).interactive()

lines  + points

alt.LayerChart(...)

### Vergleiche mit den PROD-Daten

In [56]:
prod_path = r"\\szh.loc\ssz\users\sszsim\Download\tmp"

In [57]:
prod_dfs = []
for root, dirs, files in os.walk(prod_path, topdown=True):
    for name in files:
        prod_dfs.append(pd.read_csv(
            os.path.join(root, name)
            , parse_dates = ['zeitpunkt']
        ))
print('done')  

df_prod_bruttolastgang = pd.concat(prod_dfs, ignore_index=False)
df_prod_bruttolastgang.dtypes

done


zeitpunkt         datetime64[ns]
bruttolastgang           float64
status                    object
dtype: object

In [58]:
df_prod_bruttolastgang = (
    df_prod_bruttolastgang
    .copy()
    .assign(
        #Aktualisierungs_Datum_str= lambda x: x.Aktualisierungs_Datum.astype(str),
        timestamp_str = lambda x: x.zeitpunkt.astype(str),
        timestamp_str_ISO = lambda x: x.zeitpunkt.dt.strftime("%Y-%m-%dT%H:%M:%S"),        
        day = lambda x: x.zeitpunkt.dt.strftime("%Y-%m-%d"),
        day_of_year = lambda x: x.zeitpunkt.dt.day_of_year,      
        day_nr = lambda x: x.zeitpunkt.dt.day,
        weekday = lambda x: x.zeitpunkt.dt.dayofweek,
        weekday_name = lambda x: x.zeitpunkt.dt.day_name(),
        year = lambda x: x.zeitpunkt.dt.year,
        month = lambda x: x.zeitpunkt.dt.month
    )
    .sort_values(['status','zeitpunkt'], ascending=[False, False])
    )
df_prod_bruttolastgang.columns
#data2betested.dtypes
df_prod_bruttolastgang.head(2)

,zeitpunkt,bruttolastgang,status,timestamp_str,timestamp_str_ISO,day,day_of_year,day_nr,weekday,weekday_name,year,month
24764,2022-09-16 00:00:00,58215.06284,F,2022-09-16 00:00:00,2022-09-16T00:00:00,2022-09-16,259,16,4,Friday,2022,9
24763,2022-09-15 23:45:00,59446.27685,F,2022-09-15 23:45:00,2022-09-15T23:45:00,2022-09-15,258,15,3,Thursday,2022,9


In [59]:
df_prod_bruttolastgang = df_prod_bruttolastgang.set_index("zeitpunkt")

In [60]:
#myAgg = data2betested.loc["2008-11-30":"2021-10-31"]
myAgg_prod = df_prod_bruttolastgang.loc["2019-01-01":date_today]\
    .groupby(['status','day','weekday', 'weekday_name', 'year', 'month']) \
    .agg(sum_bruttolastgang =('bruttolastgang', 'sum')
         , cnt_bruttolastgang =('bruttolastgang', 'count')
        , avg_bruttolastgang=('bruttolastgang', 'mean')
        )\
    .sort_values('day', ascending=False) 

myAgg_prod.reset_index().head(6)

C:\ProgramData\Anaconda3\envs\gogo_gis\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  


,status,day,weekday,weekday_name,year,month,sum_bruttolastgang,cnt_bruttolastgang,avg_bruttolastgang
0,F,2022-09-16,4,Friday,2022,9,58215.06284,1,58215.06284
1,F,2022-09-15,3,Thursday,2022,9,7402019.66346,96,77104.37149
2,F,2022-09-14,2,Wednesday,2022,9,7543459.12002,96,78577.69917
3,F,2022-09-13,1,Tuesday,2022,9,7359741.22482,96,76663.97109
4,F,2022-09-12,0,Monday,2022,9,6863161.32953,96,71491.26385
5,F,2022-09-11,6,Sunday,2022,9,5808325.95433,96,60503.39536


In [61]:
myTitle="PROD Bruttolastgang (SAS)"

highlight = alt.selection(type='single', on='mouseover',
                          fields=['sum_bruttolastgang'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(myAgg_prod.reset_index().query('year > 10'), title=myTitle).encode(
    x=alt.X('day:T', axis=alt.Axis(title='Wochentag (0=Mo, 6=So)'))# , axis=alt.Axis(format='%', title='percentage')
    , y=alt.X('avg_bruttolastgang', axis=alt.Axis(title='Bruttolastgang'))
    , color=alt.Color('status', legend=alt.Legend(title="Status", orient="right"))  
    ,tooltip=['status', 'day', 'weekday', 'weekday_name', 'year', 'month', 'sum_bruttolastgang']    
)
points = base.mark_circle().encode(
    opacity=alt.value(0.75)
).add_selection(
    highlight
).properties(
    width=750 , height=350
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(0.5), alt.value(4))
).interactive()

lines  + points

alt.LayerChart(...)

#### Spezialauswertung Messdaten seit Anfangs Jahr
Evt für die Zahl der Woche

In [62]:
#yearly_sums = data2betested[['pedestrians_count', 'unverified','adult_pedestrians_count', 'child_pedestrians_count']].resample("Y").sum()
yearly_sums

,bruttolastgang
timestamp,
2019-12-31,2741078542.88551
2020-12-31,2719754214.54554
2021-12-31,2637316640.55692
2022-12-31,1987740809.86805


Die Summation betrachtet die Variable `unverified` nicht. Daher ist nicht klar, ob diese Aussage genügt. Im Folgenden mache ich eine Aggregation.

In [63]:
data2betested.columns

Index(['bruttolastgang', 'status', 'timestamp_str', 'timestamp_str_ISO', 'day', 'day_of_year',
       'day_nr', 'weekday', 'weekday_name', 'year', 'month'],
      dtype='object')

------ stop testing

In [64]:
#aggYearlySums = data2betested.loc["2022-01-01":date_today]\
#    .groupby(['year','location_id','location_name', 'unverified']) \

aggYearlySums = data2betested\
    .groupby(['year','unverified']) \
    .agg(sum_all_pedestrians_count=('pedestrians_count', 'sum')
         , sum_ltr_pedestrians_count=('ltr_pedestrians_count', 'sum')
         , sum_rtl_pedestrians_count=('rtl_pedestrians_count', 'sum')
        , avg_temp=('temperature', 'mean')
        )\
    .sort_values('year', ascending=False) 

aggYearlySums.reset_index().head(6)

KeyError: 'unverified'

Die unverifizierten Zählungen machen nur einen sehr sehr geringen Wert aus. Daher kann wohl auch mit den resamples gearbeitet werden.

#### Barcharts mit Seaborn

In [ ]:
#myAgg = data2betested.loc["2008-11-30":"2021-10-31"]
myAggBar = data2betested.loc["2022-06-01":date_today]\
    .groupby(['weekday','weekday_name','weather_condition']) \
    .agg(sum_ltr_pedestrians_count=('ltr_pedestrians_count', 'sum')
         , sum_rtl_pedestrians_count=('rtl_pedestrians_count', 'sum')
         , sum_all_pedestrians_count=('pedestrians_count', 'sum')
        , avg_temp=('temperature', 'mean')
        )\
    .sort_values('sum_ltr_pedestrians_count', ascending=False) 

myAggBar.reset_index().head(6)

In [ ]:
#help(sns.catplot)     
#data2betested.columns

In [ ]:
#Stile for the following sns graphs
sns.set_theme(style="whitegrid")

In [ ]:
#sns.color_palette("flare", as_cmap=True)
myHist = sns.catplot(x="weekday"
            , y="sum_ltr_pedestrians_count"
            , hue="weather_condition"
            , kind="bar" #boxen, violin, --> wenn die aggregation nicht genau den werten in der grafik entspricht
            , palette="pastel" #, color="green"
            , height=5
            , aspect=2
            , order=None, legend_out=True
            ,data=myAggBar.reset_index()
           )
myHist.set_xticklabels(rotation=0) 
myHist.set_xlabels('Wochentag', fontsize=11) # not set_label
myHist.set_ylabels('Anz. Pers.', fontsize=11)


In [ ]:
#myAgg = data2betested.loc["2022-01-01":date_today].query('unverified == True')\

myAggBar = data2betested.loc["2022-08-13 03":"2022-08-14 08"]\
    .groupby(['location_id', 'location_name','ltr_label', 'rtl_label']) \
    .agg(sum_ltr_pedestrians_count=('ltr_pedestrians_count', 'sum')
         , sum_rtl_pedestrians_count=('rtl_pedestrians_count', 'sum')
         , sum_all_pedestrians_count=('pedestrians_count', 'sum')
        , avg_temp=('temperature', 'mean')
        , sum_adult_pedestrians_count=('adult_pedestrians_count', 'sum')
         , sum_child_pedestrians_count=('child_pedestrians_count', 'sum')         
        )\
    .sort_values('sum_ltr_pedestrians_count', ascending=False) 

myAggBar.reset_index().head(3)

In [ ]:
monthly_sums.reset_index().columns

In [ ]:
#aktuell nimmt die Zeitreihe nicht das laufende Jahr auf.

#days_tb = df_zeitreihe_tb.loc["2017-01-01 00:00":"2021-12-31 00:00"].dropna(axis=1).resample("D").median()
#months_tb =df_zeitreihe_tb[['air_temperature','water_temperature']].dropna(axis=1).loc["2017-01-01 00:00":"2021-12-31 00:00"].resample("M").median()


brush = alt.selection(type='interval', encodings=['x'])

upper = alt.Chart(weekly_sums[['zone_1_pedestrians_count', 'zone_2_pedestrians_count', 'zone_3_pedestrians_count']].reset_index().melt("timestamp")).mark_area(interpolate="basis").encode(
    x = alt.X('timestamp:T', axis=None),
    y = alt.Y('value:Q', axis=None),
    color='variable'
).properties(width=1000, height=50).add_selection(brush)

lower = alt.Chart(daily_sums[['pedestrians_count','zone_1_pedestrians_count', 'zone_2_pedestrians_count', 'zone_3_pedestrians_count']].reset_index().melt("timestamp")).mark_line(strokeWidth=1).encode(
    x = alt.X('timestamp:T', scale=alt.Scale(domain=brush)),
    y='value',
    color='variable',
).properties(width=1000, height=300)

upper & lower

In [ ]:
#data2betested.reset_index().columns

In [ ]:
#.loc["2022-01-01":"2022-04-01"]
#.loc["2022-01-01":date_today]

myAggBar2 = data2betested.loc["2022-01-01":date_today].reset_index()\
    .groupby(['timestamp','location_id', 'location_name','ltr_label', 'rtl_label']) \
    .agg(sum_all_pedestrians_count=('pedestrians_count', 'sum')
         , sum_ltr_pedestrians_count=('ltr_pedestrians_count', 'sum')
         , sum_rtl_pedestrians_count=('rtl_pedestrians_count', 'sum')
        , avg_temp=('temperature', 'mean')
        , sum_adult_pedestrians_count=('adult_pedestrians_count', 'sum')
         , sum_child_pedestrians_count=('child_pedestrians_count', 'sum')         
        )\
    .sort_values('sum_all_pedestrians_count', ascending=False) 
myAggBar2.head()


In [ ]:
#g = sns.FacetGrid(myAggBar.reset_index(), col="QuarLang", hue="HerkunftLang", col_wrap=6, height=3,)
#g.map(sns.scatterplot, "AlterV05Sort", "sum_WBev", alpha=.8)
#g.add_legend()

g2 = sns.FacetGrid(myAggBar2.reset_index(), col="location_name", hue="location_name", col_wrap=3, height=4,)
g2.map(sns.lineplot, "timestamp","sum_child_pedestrians_count",alpha=.8)

g2.set_xticklabels(rotation=45, fontsize=6) 
g2.add_legend()

g3 = sns.FacetGrid(myAggBar2.reset_index(), col="location_name", hue="location_name", col_wrap=3, height=4,)
g3.map(sns.lineplot, "timestamp","sum_adult_pedestrians_count",alpha=.8)
#g2.map(sns.lineplot, "timestamp","sum_adult_pedestrians_count",alpha=.8)

g3.set_xticklabels(rotation=45, fontsize=6) 
g3.add_legend()

In [ ]:
g4 = sns.FacetGrid(data2betested.loc["2022-07"].reset_index(),col="location_name",  hue="weather_condition", col_wrap=3, height=4,)
g4.map(sns.jointplot, "ltr_pedestrians_count", "rtl_pedestrians_count",alpha=.8)

#sns.jointplot(data=data2betested.loc["2022-07"].reset_index(), x="ltr_pedestrians_count", y="rtl_pedestrians_count", hue="weather_condition")
#sns.jointplot(data=data2betested.loc["2022-07"].reset_index(), x="ltr_pedestrians_count", y="rtl_pedestrians_count", hue="location_name")

### Daten in interaktiver Treemap zeigen
Dazu gibt es eine sehr nützliche Webseite https://plotly.com/python/treemaps/
Zu Farbskalen, siehe: https://plotly.com/python/builtin-colorscales/

#### Aktueller Bestand der WBev nach Quartier, Herkunft und Geschlecht

In [ ]:
data2betested.columns

In [ ]:
myTreemapAgg = data2betested \
    .groupby(['year', 'month','day', 'location_name']) \
    .agg(sum_pedestrians_count=('pedestrians_count', 'sum')) \
    .sort_values('sum_pedestrians_count', ascending=True) 

myTreemapAgg.reset_index().head(6)

In [ ]:
myTitle = "Passantenfrequenzen an der Bahnhofstrasse seit September 2021"

fig = px.treemap(myTreemapAgg.query('sum_pedestrians_count>0').reset_index(), path=[px.Constant(myTitle), 'year','location_name', 'month','day']
                 , values='sum_pedestrians_count'
                 , color='sum_pedestrians_count'
                 ## bei discrete scales (bei Klassen und Strings):                 
                 #, color_discrete_map={'(?)':'lightgrey','EVP':'gold', 'SP':'#FF3030', 'PdA':'#EE3B3B', 'Grüne':'#A2CD5A','GLP':'#CAFF70','Die Mitte':'orange','FDP':'#104E8B','AL':'deeppink','SVP':'forestgreen', 'FL ZÜRI':'#8B864E','Volt':'#lightblue' }                
                 ## bei continuous scales (bei Zahlenwerten):
                , color_continuous_scale='Blues'
                #, color_continuous_midpoint=np.average(df['AnzBestWir'], weights=df['AnzBestWir'])  
                 ,height=600
                 ,width=1100                 
                )
fig.update_traces(root_color="grey")
fig.update_layout(margin = dict(t=50, l=25, r=250, b=25))
fig.show()

## --- Plausi stopped here

#### Aktueller Bestand der WBev nach Quartier und Herkunft

In [ ]:
#data2betested.columns

In [ ]:
myTreemapAgg = data2betested.loc['2021']  \
    .groupby(['StichtagDatJahr', 'QuarLang', 'HerkunftLang']) \
    .agg(sum_WBev=('AnzBestWir', 'sum')) \
    .sort_values('sum_WBev', ascending=True) 

myTreemapAgg.reset_index().head(3)

In [ ]:
myTitle = "WBev nach Quartier und Herkunft, 2021"

fig = px.treemap(myTreemapAgg.reset_index(), path=[px.Constant(myTitle), 'StichtagDatJahr', 'QuarLang', 'HerkunftLang']
                 , values='sum_WBev'
                 , color='HerkunftLang'
                 ## bei discrete scales (bei Klassen und Strings):                 
                 #, color_discrete_map={'(?)':'lightgrey','EVP':'gold', 'Ausländer/in':'#FF3030', 'PdA':'#EE3B3B', 'Grüne':'#A2CD5A','GLP':'#CAFF70','Die Mitte':'orange','FDP':'#104E8B','AL':'deeppink','SVP':'forestgreen', 'FL ZÜRI':'#8B864E','Volt':'#lightblue' }         
                 , color_discrete_map={'(?)':'lightgrey','Schweizer/in':'gold', 'SP':'#FF3030' }               
                 ## bei continuous scales (bei Zahlenwerten):
                , color_continuous_scale='blues'
                #, color_continuous_midpoint=np.average(df['AnzBestWir'], weights=df['AnzBestWir'])   
                 ,height=400
                 ,width=1100
                )
fig.update_traces(root_color="grey")
fig.update_layout(margin = dict(t=50, l=25, r=250, b=25))
fig.show()

In [ ]:
#data2betested.dtypes

## ---------------------- hier Plausi beendet

**Sharepoint als gecheckt markieren!**

Record auf Sharepoint: **[Link](https://kollaboration.intranet.stzh.ch/orga/ssz-produkte/Lists/SASA_Outputs/DispForm.aspx?ID=115&ContentTypeId=0x0100988EAF029F1EFE4CA675F53C32A5D53D01006DBC563E6FBE9E4EB6FDC780799752E1)**

---------------------------------------------------------------------------

### Test: Choroplethenkarte
Importiere die Geodaten als GeoJSON

#### Stadtkreise

In [ ]:
# Read abt. Geopandas https://geopandas.org/docs/user_guide/io.html
geojson_url_kreise = "https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Stadtkreise?service=WFS&version=1.1.0&request=GetFeature&outputFormat=GeoJSON&typename=adm_stadtkreise_v"
adm_stadtkreise_v = gpd.read_file(geojson_url_kreise)

adm_stadtkreise_v.head(1)


#### Statistische Zonen

In [ ]:
# Read abt. Geopandas https://geopandas.org/docs/user_guide/io.html
geojson_url_statzonen = "https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Statistische_Zonen?service=WFS&version=1.1.0&request=GetFeature&outputFormat=GeoJSON&typename=adm_statzonen_map"
adm_statzonen_map = gpd.read_file(geojson_url_statzonen)

adm_statzonen_map.head(2)


#### Statistische Quartiere

In [ ]:
geojson_url_quartiere = "https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Statistische_Quartiere?service=WFS&version=1.1.0&request=GetFeature&outputFormat=GeoJSON&typename=adm_statistische_quartiere_map"
df_adm_statistische_quartiere_map = gpd.read_file(geojson_url_quartiere)

df_adm_statistische_quartiere_map.head(2)


### Teste LeafMap

Mehr Infos: https://leafmap.org/leafmap/

In [ ]:
m = leafmap.Map()

#m.add_geojson(geojson_url_kreise, layer_name="adm_stadtkreise_v", fill_colors=['red', 'yellow', 'green', 'orange'], info_mode=True)
m.add_gdf(df_adm_statistische_quartiere_map, layer_name='adm_statistische_quartiere_map', style={}, hover_style={}, style_callback=None, fill_colors=['black'], info_mode='on_hover', zoom_to_layer=True)
m.add_gdf(adm_stadtkreise_v, layer_name='adm_stadtkreise_v', style={}, hover_style={}, style_callback=None, fill_colors=['none'], info_mode='on_hover', zoom_to_layer=False)

m.add_legend(title='Legende', legend_dict=None, labels=None, colors=None, position='bottomright', builtin_legend=None, layer_name='adm_statistische_quartiere_map')
m

### Joins
Joine die importierten statistischen Daten des aktuellen Jahres zum Geodatensatz:

*Siehe dazu Doku zu [Geopandas](https://geopandas.org/docs/user_guide/mergingdata.html)*

In [ ]:
#data2betested.dtypes
data2betested.head()

In [ ]:
myAgg = data2betested.loc["2020"]  \
    .copy() \
    .groupby(['QuarLang', 'QuarCd']) \
    .agg(sum_AnzBestWir=('AnzBestWir', 'sum')) \
    .assign() \
    .sort_values('QuarCd', ascending=False) 

#myAgg.reset_index()


In [ ]:
# Rename attribute stznr to StatZoneSort, so the IDs have the same name
gdf_adm_statistische_quartiere_map = df_adm_statistische_quartiere_map[['geometry', 'qnr', 'qname', 'knr']].rename(columns={'qnr': 'QuarCd'})
gdf_adm_statistische_quartiere_map.head(1)

#### Leftjoin 
Matche die statistischen Daten mit den Geodaten

In [ ]:
# Merge with `merge` method on shared variable (stznr und StatZoneSort):
gdf_adm_statistische_quartiere_map_joined = gdf_adm_statistische_quartiere_map.merge(myAgg, on='QuarCd', how="left") #'right', 'inner', 'left'
gdf_adm_statistische_quartiere_map_joined.head(2)


In [ ]:
#does all match?
gdf_adm_statistische_quartiere_map_joined.isnull().sum()

In [ ]:
# Plot by StatQuartiere
# Weitere Schemes: scheme='quantiles', scheme='fisherjenks', scheme='natural_breaks',

gdf_adm_statistische_quartiere_map_joined.plot(column='sum_AnzBestWir', cmap='viridis', scheme='natural_breaks', legend=True )
gdf_adm_statistische_quartiere_map_joined.plot(column='sum_AnzBestWir', cmap='viridis', scheme='fisherjenks', legend=True )

#scheme werte: https://matplotlib.org/2.0.2/users/colormaps.html

#
#Neuere Gallerie: https://matplotlib.org/stable/gallery/index.html

In [ ]:
fig, ax = plt.subplots(1, 1)

gdf_adm_statistische_quartiere_map_joined.plot(column='sum_AnzBestWir', ax=ax, legend=True, legend_kwds={'label': "Anzahl WBev 2020",'orientation': "horizontal"})
#gdf_adm_statistische_quartiere_map_joined.plot(column='sum_AnzBestWir', ax=ax, legend=False, legend_kwds={'label': "Anzahl WBev 2020",'orientation': "horizontal"})

#gdf_adm_statistische_quartiere_map_joined.plot(column='sum_Whg_abgebrochen', cmap='viridis', scheme='natural_breaks', legend=True , ax=ax, legend_kwds={'label': "Anzahl Personen",'orientation': "horizontal"})

Noch zu prüfen, folgendes schöne Beispiel: https://docs.bokeh.org/en/latest/docs/user_guide/interaction/legends.html

Konkrete Beispiele von Bookeh in Jupyter NB: https://docs.bokeh.org/en/latest/docs/user_guide/jupyter.html

Noch zu prüfen, folgendes schöne Beispiel: https://docs.bokeh.org/en/latest/docs/user_guide/interaction/legends.html

Konkrete Beispiele von Bookeh in Jupyter NB: https://docs.bokeh.org/en/latest/docs/user_guide/jupyter.html

**Sharepoint als gecheckt markieren!**

Record auf Sharepoint: **[Link](https://kollaboration.intranet.stzh.ch/orga/ssz-produkte/Lists/SASA_Outputs/DispForm.aspx?ID=115&ContentTypeId=0x0100988EAF029F1EFE4CA675F53C32A5D53D01006DBC563E6FBE9E4EB6FDC780799752E1)**